In [ ]:
def partition(df,df_features,vo):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
                                        df[df_features], # datos
                                        df[vo],   # variable obj
                                        test_size=0.40, # partición
                                        random_state=1, # semilla
                                        shuffle=True) # elegir aleatoriamente
    

    
    return X_train, X_test, y_train, y_test
    

In [ ]:
def model_nn(X_train, X_test, y_train, random_state, activation):

        
    from sklearn.neural_network import MLPRegressor

    regr = MLPRegressor(max_iter=500000, activation=activation,random_state=random_state).fit(X_train, y_train)

    y_predict = regr.predict(X_test)
  


    return y_predict, regr


In [ ]:
def resultados(df):

    dfcom1 = y_test.reset_index()
    dfcom1_1 = df[['personal_id_x','customer_id','sbs_customer_id','taxpayer_id','segmento','gender_type']].reset_index()
    dfcom1_2 = dfcom1_1.merge(dfcom1, how='inner', on='index')

    dfcom2 = pd.concat([dfcom1_2, pd.DataFrame(y_predict, columns=["valor_pred"])],axis=1)

    dfcom2['error_est'] = dfcom2['vo_balance_amount']-dfcom2['valor_pred']
    dfcom2['p_error_est'] = abs(dfcom2['error_est']/dfcom2['vo_balance_amount'])

    dfcom3 = dfcom2['p_error_est'].sum()
    
    return dfcom2, dfcom3


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

def ftransformacion(X_train, X_test, y_train, transf): 
    if transf!='ln':
        if transf=='minmax':
            scaler = MinMaxScaler()
            scaler2 = MinMaxScaler()
        elif transf=='standard':
            scaler = StandardScaler()
            scaler2 = StandardScaler()
        elif transf=='robust':
            scaler= RobustScaler()
            scaler2= RobustScaler()
        elif transf=='boxcox':
            scaler= PowerTransformer(method='yeo-johnson')
            scaler2= PowerTransformer(method='yeo-johnson')

        mm_scaler = scaler.fit(X_train)
        X_train = mm_scaler.transform(X_train)
        X_test = mm_scaler.transform(X_test)


        mm_scaler2 = scaler2.fit(pd.DataFrame(y_train))
        y_train = mm_scaler2.transform(y_train)
    else:
        X_train = np.log(X_train)
        X_test = np.log(X_test)
        #y_train = y_train.values
        y_train = np.log(y_train).values
        mm_scaler = ''
        mm_scaler2 = ''
        

    return X_train, X_test, y_train, mm_scaler, mm_scaler2
    


def transformacion_inversa(y_predict, mm_scaler2):
    if mm_scaler2!='':
        y_predict = mm_scaler2.inverse_transform(pd.DataFrame(y_predict))
    else: 
        y_predict = np.exp(y_predict)
        #y_predict = y_predict
    
    return y_predict

In [1]:
from scipy import stats
def variablesCorrelacionadas(dfCons,variables,vo):

    matriz_correlacion = dfCons[variables].select_dtypes(include='number').corr()


    num_variables = len(matriz_correlacion)
    umbral_correlacion = 0.9
    variables_numericas_correlacionadas = []
    for reng in range(0, num_variables):
        for col in range(reng + 1, num_variables):
            if abs(matriz_correlacion.iloc[reng, col]) > umbral_correlacion:
                variables_numericas_correlacionadas.append(
                    [matriz_correlacion.columns[reng], 
                    matriz_correlacion.columns[col], 
                    matriz_correlacion.iloc[reng, col]])

    from scipy import stats
    variables_menos_corr = []
    for i in variables_numericas_correlacionadas:
        corr0 = abs(stats.pearsonr(dfCons[i[0]], dfCons[vo])[0]) 
        corr1 = abs(stats.pearsonr(dfCons[i[1]], dfCons[vo])[0])
        if corr0 > corr1:
            variables_menos_corr.append(i[1])
        else:
            variables_menos_corr.append(i[0])
            
    return variables_menos_corr


In [ ]:
def varOrdenporCorr(dfCons1,variables,vo, reverse):

    variables_corr = []
    for i in variables:
        corr0 = stats.pearsonr(dfCons1[i], dfCons1[vo])[0] 
        if abs(corr0)>0.0:
            variables_corr.append([i,corr0,abs(corr0)])
    
    variables_corr.sort(key = lambda row: row[2],reverse=reverse)
    
    varOrdenadas = [row[0] for row in variables_corr]
    
    return varOrdenadas